This notebook will demonstrate how to use the **constrained training algorithms** implemented in this toolkit.

To train a network, instantiate an algorithm, passing to it the model, the dataset, a list of `FairnessConstraint`s and the algorithm's hyperparameters.

Load and prepare data from `folktables`:

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
from folktables import ACSDataSource, ACSIncome

# load folktables data
data_source = ACSDataSource(survey_year='2018', horizon='1-Year', survey='person')
acs_data = data_source.get_data(states=["OK"], download=True)
features, labels, groups = ACSIncome.df_to_numpy(acs_data)
# split
X_train, X_test, y_train, y_test, groups_train, groups_test = train_test_split(
    features, labels, groups, test_size=0.2, random_state=42)
# scale
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# make into a pytorch dataset, remove the sensitive attribute (RAC1P)
features_train = torch.tensor(X_train, dtype=torch.float32)[:,:-1]
labels_train = torch.tensor(y_train,dtype=torch.float32)
dataset_train = torch.utils.data.TensorDataset(features_train, labels_train)

c_batch_size = 32
min_subgroup_size = 32
# For each subgroup, FairnessConstraint needs a list of indices of samples belonging to that subgroup
group_indices_train = [
    np.nonzero(groups_train == group_id)[0] for group_id in np.unique(groups_train)
    if np.count_nonzero(groups == group_id) > min_subgroup_size
]

# repeat for test set
features_test = torch.tensor(X_test, dtype=torch.float32)[:,:-1]
labels_test = torch.tensor(y_test,dtype=torch.float32)
dataset_test = torch.utils.data.TensorDataset(features_test, labels_test)
group_indices_test = [
    np.nonzero(groups_test == group_id)[0] for group_id in np.unique(groups_test)
    if np.count_nonzero(groups == group_id) > min_subgroup_size
]

print(f'Protected attribute: {ACSIncome.group}')
print(f'Number of subgroups considered: {len(group_indices_train)}')
print(f'Size of subgroups: {[len(g) for g in group_indices_train]}')

Protected attribute: RAC1P
Number of subgroups considered: 7
Size of subgroups: [10685, 794, 1213, 59, 259, 315, 997]


Let's say we want to add an equal loss constraint on the model.

We can do that by using the `FairnessConstraint` class, which will handle sampling (if possible, sampling an equal number of samples from each relevant subgroup in each minibatch), and passing it the function that will calculate the value of the constraint.

In [ ]:
from itertools import combinations
from src.constraints.constraint import FairnessConstraint
from src.constraints.constraint_fns import loss_equality

# the protected attribute is "Race" (RAC1P)
# we put a pairwise constraint on each combination of subgroups
constraint_bound = 0.05
constraints = []
for gr1, gr2 in combinations(group_indices_train, 2):
    c = FairnessConstraint(
        dataset=dataset_train,
        group_indices=[gr1, gr2],
        # subtract bound from absolute loss difference to bring constraint to form $c \leq 0$
        fn=lambda model, samples: torch.abs(loss_equality(torch.nn.BCEWithLogitsLoss(), model, samples)) - constraint_bound,
        batch_size=c_batch_size,
        seed=0
    )

    constraints.append(c)

print(f'Number of constraints: {len(constraints)}')

Number of constraints: 21


In [ ]:
# helper function to analyze model performance

def model_stats(model, features, labels, groups):
    with torch.inference_mode():
        gr_ind = list(combinations(groups, 2))
        vals = []
        acc_dif = []
        for i, c in enumerate(constraints[::2]):
            idx1, idx2 = gr_ind[i]
            val = c.eval(model, [(features[idx1], labels[idx1]), (features[idx2], labels[idx2])]) + constraint_bound
            vals.append(abs(val.numpy().item()))

            logits1 = model(features[idx1])
            logits2 = model(features[idx2])
            outs1 = torch.nn.functional.sigmoid(logits1).numpy()
            outs2 = torch.nn.functional.sigmoid(logits2).numpy()
            preds1 = (outs1.T > 0.5).astype(float)
            preds2 = (outs2.T > 0.5).astype(float) 
            acc1 = np.mean(preds1 == labels[idx1].numpy())
            acc2 = np.mean(preds2 == labels[idx2].numpy())
            acc_dif.append(abs(acc1-acc2))

        print(f'constraints (should be <= {constraint_bound}):')
        print(np.round(vals, decimals=3))
        print(f'c mean: {np.mean(vals)}')
        print(f'c min: {np.min(vals)}')
        print(f'c max: {np.max(vals)}')
        print('---')

        logits = model(features)
        outs = torch.nn.functional.sigmoid(logits).numpy()
        preds = (outs.T > 0.5).astype(float)
        acc = np.sum(preds == labels.numpy())/len(labels)
        print(f'accuracy: {acc}')
        print('accuracy abs. difference:')
        print(np.round(acc_dif, decimals=3))
        print(f'acc abs dif mean: {np.mean(acc_dif)}')
        print(f'acc abs dif min: {np.min(acc_dif)}')
        print(f'acc abs dif max: {np.max(acc_dif)}')

---
---

For comparison, let us first train a model **without constraints**.

Define a model:

In [ ]:
from torch.nn import Sequential
hsize1 = 64
hsize2 = 32
model_uncon = Sequential(
    torch.nn.Linear(features_train.shape[1], hsize1),
    torch.nn.ReLU(),
    torch.nn.Linear(hsize1, hsize2),
    torch.nn.ReLU(),
    torch.nn.Linear(hsize2, 1)
)

And start training:

In [ ]:
from torch.optim import Adam

loader = torch.utils.data.DataLoader(dataset_train, batch_size=32, shuffle=True)
loss = torch.nn.BCEWithLogitsLoss()
optimizer = Adam(model_uncon.parameters())
epochs = 50

for epoch in range(epochs):
    losses = []
    for batch_feat, batch_label in loader:
        optimizer.zero_grad()

        logit = model_uncon(batch_feat)
        loss = torch.nn.functional.binary_cross_entropy_with_logits(logit.squeeze(), batch_label)
        loss.backward()

        optimizer.step()
        losses.append(loss.item())
    print(f"Epoch: {epoch}, loss: {np.mean(losses)}")

Epoch: 0, loss: 0.4567770318327738
Epoch: 1, loss: 0.4267354179506323
Epoch: 2, loss: 0.42177021729626823
Epoch: 3, loss: 0.41923434915952384
Epoch: 4, loss: 0.4167473852368338
Epoch: 5, loss: 0.4148108066791402
Epoch: 6, loss: 0.41341322707012296
Epoch: 7, loss: 0.4125295749399811
Epoch: 8, loss: 0.4119187536915498
Epoch: 9, loss: 0.41052870083201143
Epoch: 10, loss: 0.4095372299530676
Epoch: 11, loss: 0.4085638278962246
Epoch: 12, loss: 0.40801226420860204
Epoch: 13, loss: 0.4076137725662972
Epoch: 14, loss: 0.4061356251061495
Epoch: 15, loss: 0.40524284524976145
Epoch: 16, loss: 0.4051640779445214
Epoch: 17, loss: 0.4038818087761423
Epoch: 18, loss: 0.40338663783456596
Epoch: 19, loss: 0.40176192541340633


Let's now analyze how well the **unconstrained** model does in terms of constraints:

In [ ]:
print('TRAIN')
model_stats(model_uncon, features_train, labels_train, group_indices_train)

TRAIN
constraints (should be <= 0.05):
[0.095 0.018 0.016 0.033 0.167 0.071 0.077 0.111 0.062 0.073 0.024]
c mean: 0.06775714321569963
c min: 0.016223162412643433
c max: 0.167217418551445
---
accuracy: 0.8080653038442754
accuracy abs. difference:
[0.06  0.018 0.034 0.049 0.102 0.043 0.041 0.026 0.011 0.042 0.017]
acc abs dif mean: 0.04009534022582278
acc abs dif min: 0.010863328243680859
acc abs dif max: 0.10150114015345646


In [ ]:
print('TEST')
model_stats(model_uncon, features_test, labels_test, group_indices_test)

TEST
constraints (should be <= 0.05):
[0.053 0.067 0.228 0.061 0.008 0.088 0.12  0.281 0.114 0.044 0.035]
c mean: 0.10008202357725664
c min: 0.008452504873275757
c max: 0.28118374943733215
---
accuracy: 0.7960379464285714
accuracy abs. difference:
[0.066 0.041 0.044 0.065 0.068 0.033 0.107 0.11  0.131 0.002 0.033]
acc abs dif mean: 0.06366219568118937
acc abs dif min: 0.002419842710223774
acc abs dif max: 0.1308355634849837


---
---

Now let us train the same model with one of the **constrained** training algorithms:

In [ ]:
from src.algorithms import SSLALM
from torch.nn import Sequential

hsize1 = 64
hsize2 = 32
model = Sequential(
    torch.nn.Linear(features_train.shape[1], hsize1),
    torch.nn.ReLU(),
    torch.nn.Linear(hsize1, hsize2),
    torch.nn.ReLU(),
    torch.nn.Linear(hsize2, 1)
)

optimizer = SSLALM(
    net=model,
    data=dataset_train,
    loss=torch.nn.BCEWithLogitsLoss(),
    constraints=constraints,
)

history = optimizer.optimize(max_runtime=180,batch_size=32,seed=42)

In [ ]:
print('TRAIN')
model_stats(model, features_train, labels_train, group_indices_train)

TRAIN
constraints (should be <= 0.05):
[0.025 0.015 0.006 0.015 0.034 0.032 0.01  0.031 0.01  0.009 0.007]
c mean: 0.017789791930805553
c min: 0.0057291388511657715
c max: 0.03421226143836975
---
accuracy: 0.7881113514267774
accuracy abs. difference:
[0.042 0.021 0.015 0.048 0.101 0.042 0.021 0.057 0.006 0.059 0.001]
acc abs dif mean: 0.03766711719966409
acc abs dif min: 0.0005621398199636696
acc abs dif max: 0.1012649389813639


In [ ]:
print('TEST')
model_stats(model, features_test, labels_test, group_indices_test)

TEST
constraints (should be <= 0.05):
[0.009 0.033 0.067 0.054 0.017 0.044 0.042 0.076 0.063 0.007 0.035]
c mean: 0.04064829783006148
c min: 0.007393181324005127
c max: 0.07623636722564697
---
accuracy: 0.7893415178571429
accuracy abs. difference:
[0.019 0.032 0.045 0.042 0.028 0.042 0.051 0.026 0.061 0.009 0.023]
acc abs dif mean: 0.03461236005151578
acc abs dif min: 0.00907441016333943
acc abs dif max: 0.06125483199524229
